## **Turi Sentiment Classifier on Skafos**

This notebook does the following:

1. Loads labeled text data.
2. Builds a sentiment classification model.
3. Converts the model to CoreML format and saves it to the Skafos framework.

The example below is based on [Turi Create's Text Classifier](https://apple.github.io/turicreate/docs/userguide/text_classifier/).

Execute each cell one-by-one, by selecting the cell and do one of the following:
-  Clicking the "play" button at the top of this frame.
-  Typing 'Control + Enter' or 'Shift + Enter'.



In [1]:
# Import necessary libraries
from skafossdk import *

import utilities.save_models as sm
from utilities.load_turicreate import *

In [2]:
tc = install_turicreate(timeout=500, retries=2) # check to make sure turicreate is installed
ska = Skafos() # initialize Skafos

Turi Create was already installed. It's been saved to the tc variable.
2019-01-09 19:43:39,564 - skafossdk.data_engine - INFO - Connecting to DataEngine
2019-01-09 19:43:39,703 - skafossdk.data_engine - INFO - DataEngine Connection Opened


### 1. **Load the data**
The data for this example is Yelp review data, paired with sentiment scores. The data is randomly split into train and test sets, where 80% of the data is used for training, and 20% is used for model evaluation.

In [4]:
# Load data from Turi Create's website
data = tc.SFrame('https://static.turi.com/datasets/regression/yelp-data.csv')

# Make a train-test split
train_data, test_data = data.random_split(0.8)

Downloading https://static.turi.com/datasets/regression/yelp-data.csv to /var/tmp/turicreate-jovyan/89/74789721-f2b9-427b-b291-6b7acc1be247.csv

Finished parsing file https://static.turi.com/datasets/regression/yelp-data.csv

Parsing completed. Parsed 100 lines in 3.1368 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[str,str,str,int,str,str,str,dict,int,int,int,list,str,str,float,float,str,int,int,float,str,str,float,str,int,str,int,int,int,dict]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Read 47948 lines. Lines per second: 11598.9

Finished parsing file https://static.turi.com/datasets/regression/yelp-data.csv

Parsing completed. Parsed 215879 lines in 9.09595 secs.

### 2. **Build the model**
We pass the data to the `tc.text_classifier.create` function and specify the data, target variable, features, and a few other arguments needed to properly train the model. To understand more about this specific function, check out the [Turi Create Documentation](https://apple.github.io/turicreate/docs/api/generated/turicreate.text_classifier.create.html#turicreate.text_classifier.create).

In [ ]:
# Train a sentiment classification model, this takes approximately 15 minutes using CPU
model = tc.text_classifier.create(
    dataset=train_data,
    target='stars',
    features=['text'],
    drop_stop_words=True,
    max_iterations=20
)

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.



### 3. **Save the model**
We save the model to Skafos so that you can download it into your project later without having to always retrain the model.

In [ ]:
# Export to coreml
coreml_model_name = "text_classifier.mlmodel"
res = model.export_coreml(coreml_model_name)

# Compress the model
compressed_model_name, compressed_model = sm.compress_model(coreml_model_name)

# Save to Skafos
sm.skafos_save_model(
    skafos=ska,
    model_name=compressed_model_name,
    compressed_model=compressed_model,
    permissions='public'
)